# 模型场景

按照下面的方式抽球，产生一个球的颜色的观测序列：
- (1)开始，从4个盒子里以等概率随机选取一个盒子，从这个盒子里随机抽出一个球，记录其颜色，然后放回
- (2)然后，从当前盒子随机转移到下一个盒子，规则是：如果当前盒子是盒子1，那么下一个盒子一定是盒子2，如果当前盒子是盒子2或3，那么分别以概率0.4和0.6转移到左边或右边的盒子，如果当前是盒子4，那么各以0.5的概率停留在盒子4或转移到盒子3
- (3)确定转移的盒子后，再从这个盒子里随机抽出一个球，记录其颜色，放回
- (4)如此下去，重复进行5次，得到一个球的颜色的观测序列：$$O = (红，红，白，白，红)$$

在这个过程中，观察者只能观测到球的颜色的序列，观测不到球是从哪个盒子取出的，即观测不到盒子的序列

# 隐马尔可夫模型三要素

上面的例子是一个典型的<b>隐马尔可夫模型</b>。有两个随机序列，一个是盒子的序列(状态序列)，一个是球的颜色的观测序列，前者是隐藏的，只有后者是可观测的。

隐马尔可夫模型有三要素，表示为$$\lambda = (A, B, \pi)$$
注：A为状态转移矩阵，B为观测概率分布矩阵，$\pi 为初始状态概率向量$

通过上面的例子，来分别计算下A,B和$\pi$的值

<b>状态转移概率分布矩阵：</b>
$$ A = 
 \left[
 \begin{matrix}
   0   & 1 &   0 & 0 \\
   0.4 & 0 & 0.4 & 0 \\
   0 & 0.4 &   0 & 0.6 \\
   0 &   0 & 0.5 & 0.5
  \end{matrix}
  \right]
$$
$A[ij]$表示从状态i转移到状态j的概率

<b>观测概率分布矩阵：</b>

$$ B = 
 \left[
 \begin{matrix}
   0.5 & 0.5 \\
   0.3 & 0.7 \\
   0.6 & 0.4 \\
   0.8 & 0.2
  \end{matrix}
  \right]
$$
$B[i0]$表示盒子i中取出红球的概率，$B[i1]$表示盒子i中取出白球的概率

<b>初始概率分布：</b>
$$\pi = (0.25,0.25,0.25,0.25)$$

# 隐马尔可夫模型的三个基本问题

(1) 概率计算问题

给定模型$\lambda = (A,B,\pi)$和观测序列$O = (o_1,o_2,...,0_T)$,计算在模型$\lambda$下观测模型出现的概率$P(O|\lambda)$

(2) 学习问题

已知观测序列$O = (o_1,o_2,...,0_T)$,估计模型$\lambda = (A,B,\pi)$参数，使得在该模型下观测序列概率$P(O|\lambda)$最大，用极大似然估计的方法估计参数

(3) 预测问题，也称为解码问题

已知模型$\lambda = (A,B,\pi)$和观测序列$O = (o_1,o_2,...,0_T)$，求对给定观测序列条件概率$P(O|\lambda)$最大的状态序列$I = (i_1,i_2,...,i_T)$。即给定观测序列，求最有可能的对应的状态序列

下面分别介绍针对不同问题的解决算法

# 概率计算算法

## 问题描述

给定模型$\lambda = (A,B,\pi)$和观测序列$O = (o_1,o_2,...,0_T)$,计算在模型$\lambda$下观测模型出现的概率$P(O|\lambda)$

# 学习算法

# 预测算法